In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
!pip install efficientnet_pytorch

In [0]:
## 諸々の import

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
import cv2
# from tqdm.auto import tqdm
import copy
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
# from torchsummary import summary
import gc

In [0]:
## Parameters

# resize後のサイズ
HEIGHT = 64
WIDTH = 64

# 画像を3次元にするかどうか（EfficientNetなどを使うときはTrue）
enable_3d = True

# True なら Cross Validation を実施する
# Kaggle に提出するデータを作るときは False にしてください
do_validation = True

# True なら submission.csv を生成する
create_submission = False

val_perc = 0.9  # validation set の割合（クロスバリデーション）

#epochs number
epochs = 2

#初期値として、保存した重みを使う時はこれをTrueに
load_flag = False

#Kaggleで提出するときはTrueにする
kaggle_flag = False
#loadするファイルへのパス
if kaggle_flag:
    model_path = '/kaggle/input/model_load/resnet18_epoch1_2020-03-09_14-58-43.pth'
else:
    model_path = '../trained_models/resnet18_epoch2_2020-03-09_15-30-55.pth'

In [0]:
if kaggle_flag:
    dataset_dir = '/kaggle/input/bengaliai-cv19'
    model_dir = '/kaggle/input/trained_models'
else:
    dataset_dir = '../dataset'
    model_dir = '../trained_models'

train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')


In [0]:
#モデルの設定

from preprocess import *
from save_load import *
#from model.CNN import model
#from model.efficientnet import model
from model.resnet18 import model
#from model.resnet34 import model
#from model.resnet50 import model
#from model.resnet101 import model
#from model.resnet152 import model

model = model()
model = try_gpu(model)
optimizer = optim.Adam(model.parameters())
if load_flag:
    model,optimizer,start_epoch = load_model(model,optimizer,model_path,1)
    model = try_gpu(model)
else:
    start_epoch = 1    
criterion1 = nn.CrossEntropyLoss() 

In [0]:
## train関数、test関数

def train(model, epoch, train_loader):
    model.train()
    
    size = len(train_loader.dataset)
    pred_r, pred_v, pred_c = np.zeros(size), np.zeros(size), np.zeros(size)
    true_r, true_v, true_c = np.zeros(size), np.zeros(size), np.zeros(size)
    index = 0

    for data in train_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()
        for i in range(inputs.size(0)):
            pred_r[index] = root_pred[i]
            pred_v[index] = vowel_pred[i]
            pred_c[index] = consonant_pred[i]
            true_r[index] = root_l[i]
            true_v[index] = vowel_l[i]
            true_c[index] = consonant_l[i]
            index += 1
    recall_r = recall_score(true_r, pred_r, average='macro')
    recall_v = recall_score(true_v, pred_v, average='macro')
    recall_c = recall_score(true_c, pred_c, average='macro')
    final_score = (2.*recall_r + recall_v + recall_c) / 4.

    print(f'Root Recall(train): {recall_r:.5f}')
    print(f'Vowel Recall(train): {recall_v:.5f}')
    print(f'Consonant Recall(train): {recall_c:.5f}')
    print(f'Score(train): {final_score:.5f}')
   

def test(model, test_loader):
    model.eval()

    size = len(test_loader.dataset)
    pred_r, pred_v, pred_c = np.zeros(size), np.zeros(size), np.zeros(size)
    true_r, true_v, true_c = np.zeros(size), np.zeros(size), np.zeros(size)
    index = 0
    
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs) 
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        for i in range(inputs.size(0)):
            pred_r[index] = root_pred[i]
            pred_v[index] = vowel_pred[i]
            pred_c[index] = consonant_pred[i]
            true_r[index] = root_l[i]
            true_v[index] = vowel_l[i]
            true_c[index] = consonant_l[i]
            index += 1

    recall_r = recall_score(true_r, pred_r, average='macro')
    recall_v = recall_score(true_v, pred_v, average='macro')
    recall_c = recall_score(true_c, pred_c, average='macro')
    final_score = (2.*recall_r + recall_v + recall_c) / 4.

    print(f'Root Recall(test): {recall_r:.5f}')
    print(f'Vowel Recall(test): {recall_v:.5f}')
    print(f'Consonant Recall(test): {recall_c:.5f}')
    print(f'Score(test): {final_score:.5f}')


In [9]:
## データの読み込み

X_all = np.empty((0, HEIGHT*WIDTH))
Y_root_all = np.empty((0, 168))
Y_vowel_all = np.empty((0, 11))
Y_cons_all = np.empty((0, 7))

for parq_i in range(1):
    print(f'Parquet {parq_i} を読み込み中')
    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)
    
    X = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])
    X_resized = resize(X, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8) # astype(np.uint8)をしてあげることで後で cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB) が実行できるようになる
    
    Y_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    X_all = np.append(X_all, X_resized, axis=0)
    Y_root_all = np.append(Y_root_all, Y_root, axis=0)
    Y_vowel_all = np.append(Y_vowel_all, Y_vowel, axis=0)
    Y_cons_all = np.append(Y_cons_all, Y_cons, axis=0)

    del X
    del X_resized
    del Y_root
    del Y_vowel 
    del Y_cons 
    gc.collect()

print(X_all.shape)
print(Y_root_all.shape)
print(Y_vowel_all.shape)
print(Y_cons_all.shape)

Y_all = [Y_root_all, Y_vowel_all, Y_cons_all]

trainval_dataset = MyDataset(X_all, Y_all, enable_3d=enable_3d, H=HEIGHT, W=WIDTH)

del X_all
del Y_root_all
del Y_vowel_all
del Y_cons_all
del Y_all
gc.collect()

Parquet 0 を読み込み中
Resizing raw image... / 前処理実行中…
(50210, 4096)
(50210, 168)
(50210, 11)
(50210, 7)


0

In [10]:
## 訓練ループ / Training Loop Kaggleでやる際には使わない

print('==========================')
print('Starting training.')

if do_validation:
    n_samples = len(trainval_dataset)
    train_size = int(len(trainval_dataset)*(1.0 - val_perc))
    val_size = n_samples - train_size
    print(f'train size: {train_size}, validation size: {val_size}')

    subset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

    if enable_3d:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        ))

    else:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        ))

    del trainval_dataset
    del subset
    gc.collect()

    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=0)

    for i in range(start_epoch,epochs+start_epoch):
        print(f'Epoch number {i}')
        train(model, i, train_loader)
        test(model, val_loader)
        save_model(model,optimizer,model_dir, i)

    # メモリ節約
    del train_dataset
    del val_dataset
    del train_loader
    del val_loader
    gc.collect()

else:
    n_samples = len(trainval_dataset)
    print(f'train size: {n_samples}')

    if enable_3d:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        )
    
    else:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        )

    train_loader = DataLoader(dataset=trainval_dataset, batch_size=32, shuffle=True, num_workers=4)

    for i in range(start_epoch,epochs+start_epoch):
        print(f'Epoch number {i}')
        train(model, i, train_loader)
        save_model(model,optimizer,model_dir, i)

    # メモリ節約
    del trainval_dataset
    del train_loader
    gc.collect()


Starting training.
train size: 40168, validation size: 10042
Epoch number 1
Root Recall(train): 0.33391
Vowel Recall(train): 0.72906
Consonant Recall(train): 0.71291
Score(train): 0.52745
Root Recall(test): 0.56858
Vowel Recall(test): 0.86058
Consonant Recall(test): 0.82492
Score(test): 0.70566
---saving model of epoch 1---
save finished
Epoch number 2
Root Recall(train): 0.66578
Vowel Recall(train): 0.87360
Consonant Recall(train): 0.85799
Score(train): 0.76579
Root Recall(test): 0.74778
Vowel Recall(test): 0.90334
Consonant Recall(test): 0.90818
Score(test): 0.82677
---saving model of epoch 2---
save finished


In [0]:
## 提出ファイルの作成

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8)

    for k, id in enumerate(df_test_img.index.values):
        X = X_test_resized[k]

        if enable_3d:
            X = cv2.resize(X.reshape(HEIGHT, WIDTH), (224, 224),interpolation=cv2.INTER_AREA)
            X = X.reshape(224, 224, 1)
            X = cv2.cvtColor(X, cv2.COLOR_GRAY2RGB)
            X = np.transpose(X, (2,0,1)) / 255.0
            X = X.reshape(1, 3, 224, 224) 
        
        else:
            X = X.reshape(1, 1, HEIGHT, WIDTH) / 255.0

        test_input = torch.tensor(X, dtype=torch.float)
        test_input = Variable(test_input)
        test_input = try_gpu(test_input)
        
        model.eval()
        root_o, vowel_o, consonant_o = model(test_input)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        if torch.cuda.is_available():
            root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))

        root_pred, vowel_pred, consonant_pred = root_pred.item(), vowel_pred.item(), consonant_pred.item()
        
        row_id.append(id+'_consonant_diacritic')
        target.append(consonant_pred)
        row_id.append(id+'_grapheme_root')
        target.append(root_pred)
        row_id.append(id+'_vowel_diacritic')
        target.append(vowel_pred)
    
    del df_test_img
    del X_test_resized
    gc.collect()


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

if create_submission:
    df_sample.to_csv('submission.csv',index=False)

df_sample.head(36)

Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…


,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,1
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0
